In [1]:
import pandas as pd
import catboost as cb

In [2]:
# Чтение feed_data из csv файла
feed_data = pd.read_csv(r'C:\Users\Alex\Desktop\data_for_training.csv')
print(f"Feed data shape: {feed_data.shape}")

Feed data shape: (15378483, 51)


In [3]:
# rename post_id_x to post_id 
feed_data = feed_data.rename(columns={'post_id_x': 'post_id'})

# drop post_id_y
feed_data = feed_data.drop('post_id_y', axis=1)

# drop action
feed_data = feed_data.drop('action', axis=1)

In [4]:
columns_to_remove = ['user_views', 'user_likes',
                     'component_2', '4_exp_group_views', '0_exp_group_likes', '6_exp_group_likes', 
                     '1_exp_group_views', '2_exp_group_views', 'exp_group', '0_exp_group_views', 
                     'component_4', '5_exp_group_views', '6_exp_group_views', 'component_10', 
                     'component_5', 'avg_word_length', 'component_1', 'component_3', 'component_6', 
                     'component_7', 'component_8', 'word_count', 'component_9', 'day_of_week', 
                     'sentence_count', 'punctuation_count', 'day_of_month', 'os', 
                     'source', 'year','1_exp_group_likes', '2_exp_group_likes',
                     '3_exp_group_likes', '3_exp_group_views', '4_exp_group_likes',
                     '5_exp_group_likes' , 'time_since_last_action']

feed_data = feed_data.drop(columns=columns_to_remove)

In [5]:
def sample_10_percent(group):
    frac = 0.1
    return group.sample(frac=frac)

#feed_data = feed_data.groupby('user_id', group_keys=False).apply(sample_10_percent)

In [6]:
feed_data.columns

Index(['user_id', 'post_id', 'target', 'gender', 'age', 'country', 'city',
       'topic', 'month', 'hour_of_day', 'post_views', 'post_likes'],
      dtype='object')

# Print the first few rows of the DataFrame
print(feed_data.head())

# Print the summary statistics of the DataFrame
print(feed_data.describe())

# Print the information of the DataFrame
print(feed_data.info())

# Print the number of unique users
print("Number of unique users:", feed_data['user_id'].nunique())

# Print the number of unique posts
print("Number of unique posts:", feed_data['post_id'].nunique())

# Print the distribution of the number of posts per user
print("Number of posts per user:\n", feed_data['user_id'].value_counts())


In [7]:
cat_features  = ['gender', 'age', 'country', 'city', 'exp_group', 'os', 'source', 'topic']

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

# Get a list of unique users
unique_users = feed_data['user_id'].unique()

# Split unique users into training and testing sets
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)

X_train = feed_data[feed_data['user_id'].isin(train_users)]
X_train = X_train.drop('target', axis=1)

X_test = feed_data[feed_data['user_id'].isin(test_users)]
X_test = X_test.drop('target', axis=1)

y_train = feed_data[feed_data['user_id'].isin(train_users)]['target']
y_test = feed_data[feed_data['user_id'].isin(test_users)]['target']


# Create a group ID based on the 'user_id' column

group_id_dict = {user_id: idx for idx, user_id in enumerate(unique_users)}
X_train['group_id'] = X_train['user_id'].map(group_id_dict) # map() function returns a map object(which is an iterator) of the results after applying the given function to each item of a given iterable (list, tuple etc.)
X_test['group_id'] = X_test['user_id'].map(group_id_dict)

# Sort the train and test sets by 'group_id'
X_train = X_train.sort_values(by='group_id')
y_train = y_train.loc[X_train.index] # loc() function is used to access a group of rows and columns by label(s) or a boolean array.

X_test = X_test.sort_values(by='group_id')
y_test = y_test.loc[X_test.index]

In [8]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

# Get a list of unique users
unique_users = feed_data['user_id'].unique()

# Split unique users into training and testing sets
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)

X_train = feed_data[feed_data['user_id'].isin(train_users)]
X_train = X_train.drop('target', axis=1)

X_test = feed_data[feed_data['user_id'].isin(test_users)]
X_test = X_test.drop('target', axis=1)

y_train = feed_data[feed_data['user_id'].isin(train_users)]['target']
y_test = feed_data[feed_data['user_id'].isin(test_users)]['target']

# Create a group ID based on the 'user_id' column in the training set
group_id_dict_train = {user_id: idx for idx, user_id in enumerate(train_users)}
X_train['group_id'] = X_train['user_id'].map(group_id_dict_train)

# Create a group ID based on the 'user_id' column in the test set
group_id_dict_test = {user_id: idx for idx, user_id in enumerate(test_users)}
X_test['group_id'] = X_test['user_id'].map(group_id_dict_test)

# Sort the train and test sets by 'group_id'
X_train = X_train.sort_values(by='group_id')
y_train = y_train.loc[X_train.index]

X_test = X_test.sort_values(by='group_id')
y_test = y_test.loc[X_test.index]

# Create train and test Pool objects with the 'group_id' column
train_pool = Pool(X_train.drop(columns=['user_id']), y_train, group_id=X_train['group_id'])
test_pool = Pool(X_test.drop(columns=['user_id']), y_test, group_id=X_test['group_id'])


In [9]:
# Specify the training parameters 
params = {'loss_function': 'Logloss', # suitable for binary classification
          'custom_metric': ['AUC', 'NDCG'], # evaluation metrics
          'thread_count': 16,
          'verbose': 100,
          'random_seed': 42,
          'iterations': 1000,
          'learning_rate': 0.01,
          }

# Initialize CatBoostClassifier
model = CatBoostClassifier(**params)

# Fit model
model.fit(train_pool, eval_set=test_pool)

0:	learn: 0.6844017	test: 0.6841802	best: 0.6841802 (0)	total: 615ms	remaining: 10m 14s
100:	learn: 0.3625198	test: 0.3623879	best: 0.3623879 (100)	total: 43.3s	remaining: 6m 25s
200:	learn: 0.3302596	test: 0.3301469	best: 0.3301469 (200)	total: 1m 25s	remaining: 5m 38s
300:	learn: 0.3249783	test: 0.3248456	best: 0.3248456 (300)	total: 2m 7s	remaining: 4m 56s
400:	learn: 0.3235143	test: 0.3233678	best: 0.3233678 (400)	total: 2m 49s	remaining: 4m 13s
500:	learn: 0.3228368	test: 0.3226871	best: 0.3226871 (500)	total: 3m 32s	remaining: 3m 31s
600:	learn: 0.3223864	test: 0.3222310	best: 0.3222310 (600)	total: 4m 13s	remaining: 2m 48s
700:	learn: 0.3219961	test: 0.3218416	best: 0.3218416 (700)	total: 4m 55s	remaining: 2m 6s
800:	learn: 0.3216953	test: 0.3215385	best: 0.3215385 (800)	total: 5m 37s	remaining: 1m 23s
900:	learn: 0.3214469	test: 0.3212862	best: 0.3212862 (900)	total: 6m 20s	remaining: 41.8s
999:	learn: 0.3212380	test: 0.3210762	best: 0.3210762 (999)	total: 7m 1s	remaining: 0us


In [10]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.drop(columns=['user_id']).columns

for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print(f'{name}: {score}')

age: 38.56572866588417
post_views: 19.535627294930872
post_likes: 12.487347118434892
city: 10.604408251025319
month: 6.649052249053987
country: 6.053510425135744
hour_of_day: 2.9700227173565383
gender: 1.331094677697266
post_id: 1.1585140540901286
topic: 0.6437094293089937
group_id: 0.0009851170820787826


In [11]:
y_pred_proba = model.predict_proba(test_pool)[:, 1]
# Add the prediction probabilities to the test dataset
X_test['pred_proba'] = y_pred_proba

# Group by 'user_id' and find the top 5 predicted 'post_id' for each user
top_5_posts = X_test.groupby('user_id').apply(lambda x: x.nlargest(5, 'pred_proba')['post_id'])

In [12]:
top_5_posts

user_id          
200      37          6404
         39          6697
         25          6934
         30          6661
         23          5989
                     ... 
168551   15378388    5691
         15378390    4217
         15378376    3337
         15378387    2474
         15378377     361
Name: post_id, Length: 163205, dtype: int64

In [13]:
def hitrate_at_k(top_k_posts, k=5):
    hits = 0
    user_data = top_k_posts.groupby(level=0).apply(list).to_dict()

    for user_id, top_k_posts in user_data.items():
        true_post_ids = X_test[(X_test['user_id'] == user_id) & (y_test == 1)]['post_id']
        hits += len(set(top_k_posts) & set(true_post_ids))

    return hits / len(y_test[y_test == 1])

hitrate_at_5 = hitrate_at_k(top_5_posts, k=5)
print("HitRate@5:", hitrate_at_5)


HitRate@5: 0.09007142052906575


In [14]:
# Save the model to a file
model.save_model("catboost_model.cbm")